# Texas Saftey Analysis
---
### As part of this activity, Criminal activity by each county will be analyzed and will be assigned as livablity score.

--- 
Reference and Dataset Link: https://data.texas.gov/browse?Dataset-Category_Category-Tile=Public+Safety

---

**-- Limitation : --** Only 2019 and 2020 data set is available and the liviablity score evaluations is done only for these 2 years

## Dependencies and Setup

In [1]:
# import required libiraries/packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import  pprint


In [2]:
# Import datasets csv file or import data set through API

# Study data files
safetyanalysis_FY2019_path = "Resources/Safetyanalysis_FY_2019.csv"
safetyanalysis_FY2020_path = "Resources/Safetyanalysis_FY_2020.csv"

# Read the data
safety_2019 = pd.read_csv(safetyanalysis_FY2019_path)
safety_2020 = pd.read_csv(safetyanalysis_FY2020_path)

# Create dataframe


In [15]:
# Remove duplicates in CSV

safety_2019 = safety_2019.drop_duplicates()
safety_2019.to_csv('cleaned_2019.csv', index=False)

safety_2020 = safety_2020.drop_duplicates()
safety_2020.to_csv('cleaned_2020.csv', index=False)

#filter rows from year 2017
#print(pd.to_datetime(safety_2019['Offense Date']).dt.strftime('%Y'))


df1_filtered = safety_2019[(pd.to_numeric(pd.to_datetime(safety_2019['Offense Date']).dt.strftime('%Y')) >= 2017) & ((pd.to_numeric(pd.to_datetime(safety_2019['Offense Date']).dt.strftime('%Y'))) <= 2019)]
df2_filtered = safety_2020[(pd.to_numeric(pd.to_datetime(safety_2020['Offense Date']).dt.strftime('%Y')) >= 2017) & ((pd.to_numeric(pd.to_datetime(safety_2020['Offense Date']).dt.strftime('%Y'))) <= 2020)]

display(df1_filtered)
safety_2019_selcols = pd.DataFrame(df1_filtered[['County', 'Offense', 'Offense Description', 'Offense Date']])
safety_2020_selcols = pd.DataFrame(df2_filtered[['County', 'Offense', 'Offense Description', 'Offense Date']])
display(len(safety_2019_selcols.index))
display(len(safety_2020_selcols.index))

merged_df = safety_2019_selcols.append(safety_2020_selcols)
display(len(merged_df.index))
merged_df = merged_df.drop_duplicates()
display(len(merged_df.index))

display(merged_df)
 
#merged_chunks = pd.merge(df1_filtered, df2_filtered, on='County')

#display(len(merged_chunks.index))
#display(merged_chunks)
# merged_chunks.to_csv('merged_safety_file.csv', index=False)


#df1_filtered.to_csv('df1_filtered.csv', index=False)
#df2_filtered.to_csv('df2_filtered.csv', index=False)


# Create an empty list to store the concatenated chunks
#chunksize = 10000
#merged_chunks = []
#merged_dfs = []


# read in the first CSV file in chunks and merge with second CSV file


#for df1_chunks , df2_chunks in zip(pd.read_csv('df1_filtered.csv', usecols=['County', 'Offense', 'Offense Description', 'Offense Date'], chunksize=chunksize),pd.read_csv('df2_filtered.csv', usecols=['County', 'Offense', 'Offense Description', 'Offense Date'], chunksize=chunksize))
 #   display(type(df1_chunks))
  #  display(type(df2_chunks))
   # merged_chunks = pd.merge(df1_chunks, df2_chunks, on='County')
    #merged_dfs.append(merged_chunks)
    
# Concatenate all the merged chunks into a single dataframe
#merged_df = pd.concat(merged_chunks)



# Write the merged dataframe to a new CSV file
merged_df.to_csv('merged_safety_file.csv', index=False)




,Receive Date,Receive Type,Inmate Type,Gender,Race,Age,County,Offense Code,Offense,Offense Description,Sentence Date,Offense Date,Sentence (Years)
2,1/22/2019,New Rcv,ID,M,H,78,Mitchell,36010005,Violent,INDECENCY W/CHILD SEXUAL CONTACT,12/13/2018,6/1/2018,11 to 15 Years
3,9/11/2018,New Rcv,ID,M,B,81,Tarrant,54040011,Other,DRIVING WHILE INTOXICATED 3RD OR MORE,8/9/2018,10/29/2017,3 Years
10,11/5/2018,New Rcv,ID,M,W,73,Brown,35990015,Drug,POSS CS PG 1 >=1G<4G,8/13/2018,12/27/2017,2 Years & Less
11,12/6/2018,NR-Prob-New Offense,SJ,M,H,66,Wilson,23990194,Property,"THEFT PROP >=$2,500<$30K",2/13/2018,8/21/2017,11 to 12 Months
13,4/23/2019,NR-Prob-Orig,SAFP,M,W,74,Eastland,35990015,Drug,POSS CS PG 1 >=1G<4G,NaN,6/29/2018,SAFPF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62614,6/25/2019,New Rcv,SJ,F,H,32,Milam,35990014,Drug,POSS CS PG 1 <1G,6/11/2019,2/20/2019,9 to 10 Months
62615,7/31/2019,New Rcv,ID,F,W,24,Hidalgo,13150004,Violent,AGG ASSAULT CAUSES SERIOUS BODILY INJ,7/10/2019,1/19/2019,5 Years
62616,3/4/2019,New Rcv,ID,F,W,19,Bexar,48010020,Other,EVADING ARST/DETN W/VEH,2/8/2019,3/8/2018,8 Years
62617,3/5/2019,New Rcv,SJ,M,W,44,Ellis,35990014,Drug,POSS CS PG 1 <1G,2/22/2019,7/21/2017,6 Months & Less


39236

29269

C:\Users\anith\AppData\Local\Temp\ipykernel_6748\2507962616.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = safety_2019_selcols.append(safety_2020_selcols)


68505

63342

,County,Offense,Offense Description,Offense Date
2,Mitchell,Violent,INDECENCY W/CHILD SEXUAL CONTACT,6/1/2018
3,Tarrant,Other,DRIVING WHILE INTOXICATED 3RD OR MORE,10/29/2017
10,Brown,Drug,POSS CS PG 1 >=1G<4G,12/27/2017
11,Wilson,Property,"THEFT PROP >=$2,500<$30K",8/21/2017
13,Eastland,Drug,POSS CS PG 1 >=1G<4G,6/29/2018
...,...,...,...,...
57663,Travis,Property,BURG OF BLDG,2/8/2019
57664,Milam,Drug,POSS C/S <1G,2/20/2019
57665,Burnet,Drug,FRAUD POSS CS/PRESCRIP III/IV,5/14/2018
57666,Montgomery,Property,THEFT,9/14/2019


In [97]:
# Merge into dataframe

display(df1_filtered.head())
display(df2_filtered.head())

,Receive Date,Receive Type,Inmate Type,Gender,Race,Age,County,Offense Code,Offense,Offense Description,Sentence Date,Offense Date,Sentence (Years)
2,1/22/2019,New Rcv,ID,M,H,78,Mitchell,36010005,Violent,INDECENCY W/CHILD SEXUAL CONTACT,12/13/2018,6/1/2018,11 to 15 Years
3,9/11/2018,New Rcv,ID,M,B,81,Tarrant,54040011,Other,DRIVING WHILE INTOXICATED 3RD OR MORE,8/9/2018,10/29/2017,3 Years
10,11/5/2018,New Rcv,ID,M,W,73,Brown,35990015,Drug,POSS CS PG 1 >=1G<4G,8/13/2018,12/27/2017,2 Years & Less
11,12/6/2018,NR-Prob-New Offense,SJ,M,H,66,Wilson,23990194,Property,"THEFT PROP >=$2,500<$30K",2/13/2018,8/21/2017,11 to 12 Months
13,4/23/2019,NR-Prob-Orig,SAFP,M,W,74,Eastland,35990015,Drug,POSS CS PG 1 >=1G<4G,NaN,6/29/2018,SAFPF


,Release Date,Release Type,Inmate Type,Gender,Race,Age,County,Offense Code,Offense,Offense Description,Sentence Date,Offense Date,Sentence (Years)
1,3/25/2020,DMS,ID,M,H,84,Bexar,35990016,Drug,POSS CONT SUB 4-200G,7/16/2018,7/1/2017,4 Years
5,1/17/2020,Discharge,SJ,M,B,82,Brazoria,23990196,Property,THEFT W/2 MORE PREV CONV,10/11/2019,4/24/2019,8 to 9 Months
7,11/15/2019,DMS,ID,M,B,82,Tarrant,54040011,Other,DWI,8/9/2018,10/29/2017,3 Years
22,2/20/2020,SAFPF - Prob,SAFP,M,W,75,Eastland,35990015,Drug,POSS CS PG 1 >=1G<4G,NaN,6/29/2018,SAFPF
31,9/20/2019,Parole,ID,M,B,75,Harris,54040011,Other,DWI,7/10/2018,8/4/2017,5 Years


In [68]:
# Generate and export csv output by county level percent data along with Livablity score

In [66]:
merged_df.head(5)

,Receive Date,County,Offense_x,Release Date,Offense_y
0,10/22/2018,Dallas,Property,12/23/2019,Violent
1,10/22/2018,Dallas,Property,6/17/2020,Violent
2,10/22/2018,Dallas,Property,9/13/2019,Violent
3,10/22/2018,Dallas,Property,2/9/2020,Violent
4,10/22/2018,Dallas,Property,6/15/2020,Violent
